In [1]:
!pip install gurobipy
!pip install torch_geometric
!pip install sortedcontainers

#!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric
!pip install scikit-optimize

from skopt import gp_minimize
from skopt.space import Real
import random

import numpy as np
import gurobipy as gp
from gurobipy import GRB
import torch
from sklearn.model_selection import train_test_split

import itertools
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.metrics import pairwise_distances as sklearn_pairwise_distances
import networkx as nx
 # Use "cosine" for cosine similarity
import heapq


os.environ["GRB_LICENSE_FILE"] = "gurobi (3).lic"
os.environ["GRB_WLSACCESSID"] = "f218200d-1f8d-4342-83f5-b7b2d9263751"  # Replace with your actual WLSACCESSID
os.environ["GRB_WLSSECRET"] = "528d596b-babc-4a1e-bda2-693c44f4f006"  # Replace with your actual WLSSECRET
os.environ["GRB_LICENSEID"] = "840285"  # Replace with your actual LICENSEID

from collections import Counter
from sklearn.metrics.pairwise import cosine_distances
from torch_geometric.datasets import WebKB, WikipediaNetwork, Actor

from torch_geometric.datasets import FacebookPagePage

from torch_geometric.datasets import SNAPDataset

import numpy as np
import torch
import random
import networkx as nx
from torch_geometric.datasets import WebKB
from collections import Counter, defaultdict
from sklearn.metrics import accuracy_score
from sortedcontainers import SortedSet  # Ordered set
from torch_geometric.utils import to_networkx

from torch_geometric.datasets import WikipediaNetwork
from torch_geometric.datasets import Actor

# Load the Film dataset (also known as Actor dataset)


In [2]:
from torch_geometric.datasets import LINKXDataset
#dataset = WebKB(root='data', name='Cornell')
dataset = WebKB(root='data', name='texas')
#dataset = WikipediaNetwork(root='data', name='chameleon')
#dataset = WikipediaNetwork(root='data', name='squirrel')
#dataset = WebKB(root='data', name='Wisconsin')
#dataset = Actor(root='data/Film')
root = 'data/'
#dataset = LINKXDataset(root=root, name='penn94')
data = dataset[0]
data.edge_index = torch.cat([data.edge_index, data.edge_index.flip(0)], dim=1)

# ✅ Remove duplicate edges
data.edge_index = torch.unique(data.edge_index, dim=1)

# ✅ Convert to NetworkX Graph to Find Components
G = to_networkx(data, to_undirected=True)
components = list(nx.connected_components(G))
num_components = len(components)

print(f"\n✅ Loaded  dataset with {data.num_nodes} nodes, {data.edge_index.shape[1]} edges")
print(f"   - Features: {data.x.shape[1]}")
print(f"   - Number of Classes: {len(set(data.y.numpy()))}")
print(f"   - Number of Components: {num_components}")

# ✅ Set Training Percentage
# train_percentage = 0.65

# # ✅ Select Training Nodes
# train_indices = []
# for component in components:
#     component = list(component)
#     t = max(1, int(train_percentage * len(component)))
#     sampled_nodes = np.random.choice(component, t, replace=False)
#     train_indices.extend(sampled_nodes)

# np.random.shuffle(train_indices)

# # ✅ Initialize Prediction Labels
# y_pred = np.full(data.num_nodes, -1)
# for idx in train_indices:
#     y_pred[idx] = data.y[idx].item()

# # ✅ Compute Label Distribution
# total_label_counts = Counter(data.y.numpy())
# num_labels = len(set(data.y.numpy()))
# alpha = 1  # Laplace smoothing

# dataset_label_distribution = {
#     lbl: (total_label_counts[lbl] + alpha) / (len(data.y) + alpha * num_labels)
#     for lbl in total_label_counts.keys()
# }

# train_label_counts = Counter(y_pred[train_indices])

# train_label_distribution = {
#     lbl: (train_label_counts.get(lbl, 0) + alpha) / (len(train_indices) + alpha * num_labels)
#     for lbl in dataset_label_distribution.keys()
# }

# # ✅ Precompute Neighbors for Each Node
# A = torch.zeros((data.num_nodes, data.num_nodes))
# A[data.edge_index[0], data.edge_index[1]] = 1
# A = A.numpy()
# neighbors_dict = {i: set(np.where(A[i] == 1)[0]) for i in range(data.num_nodes)}

# # ✅ Track Instances Assigned to Each Label
# class_instances = defaultdict(set)
# for idx in train_indices:
#     class_instances[data.y[idx].item()].add(idx)

# # ✅ Ordered Set for Managing Unlabeled Vertices
# ordered_set = SortedSet()
# train_labeled_nodes = set(train_indices)

# for node in range(data.num_nodes):
#     if y_pred[node] == -1:  # Only process unlabeled nodes
#         labeled_neighbors = [n for n in neighbors_dict[node] if y_pred[n] != -1]
#         num_labeled_by_train = sum(1 for n in labeled_neighbors if n in train_labeled_nodes)
#         num_labeled_by_propagation = len(labeled_neighbors) - num_labeled_by_train
#         total_neighbors = len(neighbors_dict[node])

#         if total_neighbors > 0:
#             weighted_score = (num_labeled_by_propagation + 3 * num_labeled_by_train) / total_neighbors
#         else:
#             weighted_score = 0

#         ordered_set.add((-weighted_score, node))

# ✅ Iteratively Label the Most Constrained Nodes
# while ordered_set:
#     _, node = ordered_set.pop(0)

#     if y_pred[node] != -1:
#         continue

#     # Get labeled neighbors
#     neighbor_labels = [y_pred[n] for n in neighbors_dict[node] if y_pred[n] != -1]
#     label_counts = Counter(neighbor_labels)

#     # 🚨 If this node has 0 labeled neighbors
#     if not neighbor_labels:
#         print(f"\n🚨 STRANGE: Node {node} was chosen, but it has 0 labeled neighbors!")
#         break

#     # Compute Neighbor Label Distribution
#     neighbor_label_counts = Counter(neighbor_labels)
#     total_labeled_neighbors = len(neighbor_labels)

#     if total_labeled_neighbors > 0:
#         neighbor_label_distribution = {
#             lbl: neighbor_label_counts[lbl] / total_labeled_neighbors for lbl in neighbor_label_counts.keys()
#         }
#     else:
#         neighbor_label_distribution = {lbl: 0 for lbl in dataset_label_distribution.keys()}

    # ✅ Compute Feature Distance to Each Class
    # feature_diffs = {}
    # for lbl in dataset_label_distribution.keys():
    #   if class_instances[lbl]:  # Only compute if we have instances of the label
    #       instance_features = data.x[list(class_instances[lbl])]
    #       node_feature = data.x[node].unsqueeze(0)

    #       # Compute distances for each instance
    #       distances = torch.norm(instance_features - node_feature, dim=1).tolist()

    #       # Apply weight based on whether the instance is from training or predicted labels
    #       weighted_distances = [
    #           (3 * dist) if instance in train_indices else dist  # Give 3x weight if in training set
    #           for instance, dist in zip(class_instances[lbl], distances)
    #       ]

    #       # Compute weighted average distance
    #       avg_distance = sum(weighted_distances) / len(weighted_distances)
    #       feature_diffs[lbl] = avg_distance
    #   else:
    #       feature_diffs[lbl] = 0  # No penalty if no instances exist

#     feature_diffs = {}
#     for lbl in dataset_label_distribution.keys():
#         if class_instances[lbl]:  # Only compute if we have instances of the label
#             instance_features = data.x[list(class_instances[lbl])]
#             node_feature = data.x[node].unsqueeze(0)
#            # print(len(instance_features),len(node_feature))
#             avg_distance = torch.mean(torch.norm(instance_features - node_feature, dim=1)).item()
#             feature_diffs[lbl] = avg_distance
#         else:
#             feature_diffs[lbl] = 0  # No penalty if no instances exist

#     # ✅ Select the Best Label Based on the Adjusted Score
#     best_label_candidates = []
#     max_value = float('-inf')
#     a1=0.5
#     a2=-6
#     a3=-2
#     a4=1
#     a5=3
#     for lbl in dataset_label_distribution.keys():
#         score = (
#             a1*train_label_distribution.get(lbl, 0)
#             +a2* neighbor_label_distribution.get(lbl, 0)
#             +a3*feature_diffs[lbl]  # Penalize by average feature distance
#         )
#         print(lbl,score,)

#         if score - max_value > 0.01:
#             best_label_candidates = [lbl]
#             max_value = score
#         elif abs(score - max_value) <= 0.01:
#             best_label_candidates.append(lbl)


#     best_label = random.choice(best_label_candidates)
#     y_pred[node] = best_label
#     class_instances[best_label].add(node)  # Track newly labeled nodes
#     neighbor_labels = [y_pred[n] for n in neighbors_dict[node] if y_pred[n] != -1]
#     label_counts = Counter(neighbor_labels)

#     print(f"\n🔹 Labelling Node {node} ")
#     # ✅ Print Prediction Outcome
#     true_label = data.y[node].item()
#     correct = "✅" if best_label == true_label else "❌"
#     print(f"   -> Predicted Label: {best_label} | True Label: {true_label} {correct}")

#    # print(f"   - Total Neighbors: {len(neighbors_dict[node])}")
#    # print(f"   - Labeled Neighbors: {len(neighbor_labels)}")
#     for lbl, count in label_counts.items():
#         print(f"   - Class {lbl}: {count} neighbors")
#         # ✅ Update Labeled Neighbor Score for Unlabeled Neighbors
#         for neighbor in neighbors_dict[node]:
#             if y_pred[neighbor] == -1:
#                 old_entry = next((entry for entry in ordered_set if entry[1] == neighbor), None)

#                 if old_entry:
#                     ordered_set.discard(old_entry)

#                 num_labeled_by_train = sum(1 for n in neighbors_dict[neighbor] if y_pred[n] != -1 and n in train_labeled_nodes)
#                 num_labeled_by_propagation = sum(1 for n in neighbors_dict[neighbor] if y_pred[n] != -1 and n not in train_labeled_nodes)
#                 total_neighbors = len(neighbors_dict[neighbor])

#                 new_weighted_score = (a4*num_labeled_by_propagation + a5 * num_labeled_by_train) / total_neighbors if total_neighbors > 0 else 0

#                 ordered_set.add((-new_weighted_score, neighbor))
#     print("\n\n\n")

# # ✅ Evaluate Accuracy
# test_indices = [i for i in range(data.num_nodes) if i not in train_indices]
# y_true = data.y[test_indices]
# y_pred_test = y_pred[test_indices]

# valid_idx = [i for i in range(len(y_pred_test)) if y_pred_test[i] != -1]
# if valid_idx:
#     final_accuracy = accuracy_score(y_true[valid_idx], y_pred_test[valid_idx])
#     print(f"\n🎯 Label Propagation Accuracy: {final_accuracy:.4f}")
# else:
#     print("\n❌ No valid predictions were made!")


Processing...
Done!



✅ Loaded  dataset with 183 nodes, 574 edges
   - Features: 1703
   - Number of Classes: 5
   - Number of Components: 1


In [3]:
import torch
import numpy as np
import random
from collections import Counter, defaultdict
from sortedcontainers import SortedSet
from sklearn.metrics import accuracy_score
import torch.nn.functional as F

import torch
import numpy as np
import random
from collections import Counter, defaultdict
from sortedcontainers import SortedSet
from sklearn.metrics import accuracy_score
import torch.nn.functional as F

def predictclass(data, train_indices, test_indices, a1, a2, a3, a4, a5, a6, a7):
   # print(f"data.x shape: {data.x.shape}")
    data.edge_index = torch.cat([data.edge_index, data.edge_index.flip(0)], dim=1)
    train_indices = list(train_indices)
    test_indices = list(test_indices)

    # ✅ Remove duplicate edges
    data.edge_index = torch.unique(data.edge_index, dim=1)

    # ✅ Initialize Prediction Labels
    y_pred = np.full(data.num_nodes, -1, dtype=int)
    for idx in train_indices:
        y_pred[idx] = data.y[idx].item()

    # ✅ Compute Label Distribution
    total_label_counts = Counter(data.y.numpy())
    num_labels = len(set(data.y.numpy()))
    alpha = 1  # Laplace smoothing

    dataset_label_distribution = {
        lbl: (total_label_counts[lbl] + alpha) / (len(data.y) + alpha * num_labels)
        for lbl in total_label_counts.keys()
    }

    train_label_counts = Counter(y_pred[train_indices])

    train_label_distribution = {
        lbl: (train_label_counts.get(lbl, 0) + alpha) / (len(train_indices) + alpha * num_labels)
        for lbl in dataset_label_distribution.keys()
    }

    # ✅ Precompute Neighbors for Each Node
    A = torch.zeros((data.num_nodes, data.num_nodes))
    A[data.edge_index[0], data.edge_index[1]] = 1
    neighbors_dict = {i: set(np.where(A[i] == 1)[0]) for i in range(data.num_nodes)}

    # ✅ Track Instances Assigned to Each Label
    class_instances = defaultdict(set)
    for idx in train_indices:
        class_instances[data.y[idx].item()].add(idx)

    # ✅ Initialize D Table (Similarity Table)
    D = {i: {lbl: 0.0 for lbl in dataset_label_distribution.keys()} for i in range(data.num_nodes)}
    class_counts = {lbl: 0 for lbl in dataset_label_distribution.keys()}

    # ✅ Compute Average Feature Vector Once Per Label
    avg_features = {}
    for lbl in dataset_label_distribution.keys():
        if class_instances[lbl]:
            instance_list = list(map(int, class_instances[lbl]))  # Ensure list of Python ints
            instance_list = torch.tensor(instance_list, dtype=torch.long)  # Convert to PyTorch tensor

            instance_features = data.x[instance_list]
            instance_weights = torch.tensor(
                [(1 - a7) if inst in train_indices else a7 for inst in instance_list],
                dtype=torch.float,
                device=data.x.device
            )
            instance_weights /= instance_weights.sum()
            avg_features[lbl] = torch.sum(instance_features * instance_weights.view(-1, 1), dim=0, keepdim=True)

    # ✅ Compute Similarity for Each Unlabeled Node
    for lbl in dataset_label_distribution.keys():
        if lbl in avg_features:
            for i in range(data.num_nodes):
                similarity = F.cosine_similarity(data.x[i].unsqueeze(0), avg_features[lbl]).item()
                D[i][lbl] = (similarity + 1) / 2

    # ✅ Ordered Set for Managing Unlabeled Vertices
    ordered_set = SortedSet()
    train_labeled_nodes = set(train_indices)
    labeled_nodes_count = 0

    for node in range(data.num_nodes):
        if y_pred[node] == -1:
            labeled_neighbors = [n for n in neighbors_dict[node] if y_pred[n] != -1]
            num_labeled_by_train = sum(1 for n in labeled_neighbors if n in train_labeled_nodes)
            num_labeled_by_propagation = len(labeled_neighbors) - num_labeled_by_train
            total_neighbors = len(neighbors_dict[node])
            weighted_score = (a4 * num_labeled_by_propagation + a5 * num_labeled_by_train) / total_neighbors if total_neighbors > 0 else 0
            similarity_score = max(D[node].values()) if D[node] else 0
            ordered_set.add((-weighted_score - a6 * similarity_score, node))

    while ordered_set:
        _, node = ordered_set.pop(0)

        if y_pred[node] != -1:
            continue
            print("This node is already labeled. Something is wrong.")

        neighbor_labels = [y_pred[n] for n in neighbors_dict[node] if y_pred[n] != -1]


        neighbor_label_counts = Counter(neighbor_labels)
        total_labeled_neighbors = len(neighbor_labels)
        neighbor_label_distribution = {
            lbl: neighbor_label_counts[lbl] / total_labeled_neighbors for lbl in neighbor_label_counts.keys()
        } if total_labeled_neighbors > 0 else {lbl: 0 for lbl in dataset_label_distribution.keys()}

        label_similarity = {lbl: D[node][lbl] for lbl in dataset_label_distribution.keys()}

        best_label_candidates = []
        max_value = float('-inf')

        for lbl in dataset_label_distribution.keys():
            score = (
                a1 * train_label_distribution.get(lbl, 0)
                + a2 * neighbor_label_distribution.get(lbl, 0)
                + a3 * label_similarity[lbl]
            )

            if score - max_value > 0.01:
                best_label_candidates = [lbl]
                max_value = score
            elif abs(score - max_value) <= 0.01:
                best_label_candidates.append(lbl)

        best_label = random.choice(best_label_candidates)
        y_pred[node] = best_label
        class_instances[best_label].add(node)
        class_counts[best_label] += 1
        labeled_nodes_count += 1



        if labeled_nodes_count % (data.num_nodes // 5) == 0 == 0:
            avg_features = {}
            for lbl in dataset_label_distribution.keys():
                if class_instances[lbl]:
                    instance_list = list(class_instances[lbl])
                    instance_list = list(map(int, class_instances[lbl]))  # Ensure list of Python ints
                    instance_list = torch.tensor(instance_list, dtype=torch.long)  # Convert to PyTorch tensor

                    instance_features = data.x[instance_list]  # Now indexing should work

                    instance_weights = torch.tensor(
                        [(1 - a7) if inst in train_indices else a7 for inst in instance_list],
                        dtype=torch.float,
                        device=data.x.device
                    )
                    instance_weights /= instance_weights.sum()
                    avg_features[lbl] = torch.sum(instance_features * instance_weights.view(-1, 1), dim=0, keepdim=True)

            for lbl in dataset_label_distribution.keys():
                if lbl in avg_features:
                    for i in range(data.num_nodes):
                        similarity = F.cosine_similarity(data.x[i].unsqueeze(0), avg_features[lbl]).item()
                        D[i][lbl] = (similarity + 1) / 2

            # ✅ Update Scores in Ordered Set
            new_ordered_set = SortedSet()
            for node in range(data.num_nodes):
                if y_pred[node] == -1:
                    labeled_neighbors = [n for n in neighbors_dict[node] if y_pred[n] != -1]
                    num_labeled_by_train = sum(1 for n in labeled_neighbors if n in train_labeled_nodes)
                    num_labeled_by_propagation = len(labeled_neighbors) - num_labeled_by_train
                    total_neighbors = len(neighbors_dict[node])
                    weighted_score = (a4 * num_labeled_by_propagation + a5 * num_labeled_by_train) / total_neighbors if total_neighbors > 0 else 0
                    similarity_score = max(D[node].values()) if D[node] else 0
                    new_ordered_set.add((-weighted_score - a6 * similarity_score, node))
            ordered_set = new_ordered_set

    return torch.tensor(y_pred[test_indices], dtype=torch.float), accuracy_score(data.y[test_indices], y_pred[test_indices])


In [4]:
label_shift = -data.y.min().item() if data.y.min().item() < 0 else 0
y_shifted = data.y + label_shift


data = dataset[0]



# Count occurrences of each class label
class_counts = torch.bincount(y_shifted)

# Print the number of elements in each class
for class_id, count in enumerate(class_counts):
    print(f"Class {class_id}: {count} elements")
a1=0.5
a2=-6
a3=-2
a4=1
a5=3
a6=1

indices = np.arange(data.num_nodes)  # NumPy array of indices [0, 1, 2, ..., num_nodes-1]
np.random.shuffle(indices)
train_idx, test_idx = train_test_split(indices, test_size=0.6)

# Further split train_idx into train and validation sets (e.g., 20% of train goes to validation)
train_idx, val_idx = train_test_split(train_idx, test_size=0.5)
test_indices=test_idx



Class 0: 33 elements
Class 1: 1 elements
Class 2: 18 elements
Class 3: 101 elements
Class 4: 30 elements


In [5]:
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from sklearn.model_selection import KFold
import warnings
warnings.simplefilter("error", RuntimeWarning)  # Convert warnings to errors
#dataset = WebKB(root='data', name='Cornell')
#dataset = WebKB(root='data', name='texas')
#dataset = WikipediaNetwork(root='data', name='chameleon')
#dataset = WikipediaNetwork(root='data', name='squirrel')
#dataset = WebKB(root='data', name='Wisconsin')



def random_search(model, data, num_samples=50, num_splits=5):
    """
    Perform random search for hyperparameter tuning while evaluating each set
    of hyperparameters across multiple train-test splits.

    Arguments:
        model: Function to predict classes.
        data: Dataset object.
        num_samples: Number of random search samples (default: 120).
        num_splits: Number of different train-test splits to evaluate each hyperparameter set.

    Returns:
        best_params: The best hyperparameters found based on average accuracy.
        best_score: The highest average accuracy obtained.
    """
    best_params = None
    best_score = float('-inf')
    indices = np.arange(data.num_nodes)
    maxi=1
    for _ in range(num_samples):
        print("step number ",_)
        # Generate random hyperparameters
        a1 = random.uniform(0, maxi)     # a1 must be positive
        a2 = random.uniform(-maxi, -0.01)  # a2 must be negative
        a3 = random.uniform(0, maxi)  # a3 must be negative
        a5 = random.uniform(0, maxi)     # a5 must be positive
        a4 = random.uniform(0, a5)    # a4 must be positive and a4 <= a5
        a6 = random.uniform(0, maxi)
        a7 = random.uniform(0, maxi/2)
       # a8 = random.uniform(0, a7)
        params = [a1, a2, a3, a4, a5, a6,a7]

        fold_scores = []
        kf = KFold(n_splits=num_splits, shuffle=True, random_state=None)  # Different train-test splits

        for train_idx, val_idx in kf.split(indices):
            _, score = model(data, train_idx, val_idx, *params)
            fold_scores.append(score)

        avg_score = np.mean(fold_scores)

        if avg_score > best_score:
            best_score = avg_score
            best_params = params

    return best_params, best_score




# Initial guesses for a1, a2, a3, a4, a5
#initial_params = [0.5,-5,-2,1,3]

# Bounds for parameters
 # Adjust bounds as needed

# Optimize using scipy
# result,best_val = random_search(predictclass,data,num_samples=15,num_splits=5)
# #print(result)
# # Get best parameters
# best_a1, best_a2, best_a3, best_a4, best_a5, best_a6,best_a7 = result[0],result[1],result[2],result[3],result[4],result[5],result[6]

# # Evaluate on test set


# # Print results
# print("best validation by random search:", best_val)


In [6]:


from skopt.space import Real
from skopt import gp_minimize
def bayesian_optimization(model, data, num_calls=10, num_splits=5, initial_params=None):
    """
    Bayesian Optimization to find best hyperparameters (a1, a2, a3, a4, a5, a6)
    that maximize the validation accuracy while evaluating across multiple train-test splits.

    Arguments:
        model: Function that takes (data, train_idx, val_idx, a1, a2, a3, a4, a5, a6) and returns (loss, accuracy)
        data: Dataset object.
        num_calls: Number of function evaluations (default: 30).
        num_splits: Number of different train-test splits to evaluate each hyperparameter set.
        initial_params: Optional initial solution from random search.

    Returns:
        best_params: The best found values for [a1, a2, a3, a4, a5, a6].
        best_accuracy: The best validation accuracy found.
    """
    maxi=1
    search_space = [
        Real(0, maxi, name="a1"),        # a1 must be positive
        Real(-maxi, -0.01, name="a2"),   # a2 must be negative
        Real(0, maxi, name="a3"),   # a3 must be negative
        Real(0, maxi, name="a4"),        # a4 must be positive and a4 ≤ a5
        Real(0, maxi, name="a5"),        # a5 must be positive
        Real(0, maxi, name="a6"),
        Real(0, maxi/2, name="a7") , # a6 must be positive
       # Real(0, maxi, name="a8")
    ]

    indices = np.arange(data.num_nodes)

    def objective(params):
        a1, a2, a3, a4, a5, a6,a7 = params
        a4, a5 = min(a4, a5), max(a4, a5)  # Ensure a4 ≤ a5
       # a8, a7 = min(a8, a7), max(a8, a7)
        fold_scores = []
        kf = KFold(n_splits=num_splits, shuffle=True, random_state=None)  # Different train-test splits

        for train_idx, val_idx in kf.split(indices):
            _, score = model(data, train_idx, val_idx, a1, a2, a3, a4, a5, a6,a7)
            fold_scores.append(score)

        avg_score = np.mean(fold_scores)
        return -avg_score  # Since `gp_minimize` minimizes the function, we negate accuracy

    if initial_params is not None:
        print("initial_params=",initial_params)
        initial_params[3], initial_params[4] = min(initial_params[3], initial_params[4]), max(initial_params[3], initial_params[4])

        #initial_params[7], initial_params[6] = min(initial_params[7], initial_params[6]), max(initial_params[7], initial_params[6])
        result = gp_minimize(objective, search_space, n_calls=num_calls, x0=[initial_params], random_state=None)
    else:
        result = gp_minimize(objective, search_space, n_calls=num_calls, random_state=None)

    best_a1, best_a2, best_a3, best_a4, best_a5, best_a6,best_a7 = result.x
    best_a4, best_a5 = min(best_a4, best_a5), max(best_a4, best_a5)  # Ensure a4 ≤ a5
    #best_a8, best_a7 = min(best_a8, best_a7), max(best_a8, best_a7)
    best_accuracy = -result.fun

    best_params = [best_a1, best_a2, best_a3, best_a4, best_a5, best_a6,best_a7]

    return best_params, best_accuracy

# Example Usage:




In [7]:

test_accuracies = []

# Number of iterations
num_iterations = 10
for run in range(num_iterations):
    print(f"\n🔄 Iteration {run + 1}/{num_iterations}")

    indices = np.arange(data.num_nodes)

# Step 1: Split into 80% train+validation and 20% test
    train_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=run, shuffle=True)


    train_idx = torch.tensor(train_idx, dtype=torch.long).contiguous()

    test_idx = torch.tensor(test_idx, dtype=torch.long).contiguous()


    # Now call subgraph()
    train_data = data.subgraph(train_idx)
    test_data = data.subgraph(test_idx)



    # Convert indices to PyTorch tensors for PyG subgraph function
    #train_idx = torch.tensor(train_idx, dtype=torch.long)
    #test_idx = torch.tensor(test_idx, dtype=torch.long)


    # Step 2: Split train+validation into 75% train and 25% validation
    #train_idx, val_idx = train_test_split(train_val_idx, test_size=0.25, random_state=run)
    # (0.25 * 80%) = 20% of total, so remaining 60% is train

    # Now we have:
    # - train_idx (60% of total)
    # - val_idx (20% of total)
    # - test_idx (20% of total)

    # Run hyperparameter tuning
    best_params_random, best_val_random = random_search(predictclass, train_data, num_samples=10, num_splits=5)

    best_params_bayes, best_val_bayes = bayesian_optimization(predictclass, train_data, num_calls=20, initial_params=best_params_random)

    best_params = best_params_bayes if best_val_bayes >= best_val_random else best_params_random

    # Extract optimized hyperparameters
    best_a1, best_a2, best_a3, best_a4, best_a5, best_a6, best_a7 = best_params
    #print("best params found")
    #print(len(data),len(train_idx),len(test_idx))
    #print(data)
    # Run predictclass with best parameters on test set
    _, final_test_accuracy = predictclass(data, train_idx, test_idx, best_a1, best_a2, best_a3, best_a4, best_a5, best_a6, best_a7)

    # Store the result
    test_accuracies.append(final_test_accuracy)

    print(f"🎯 Test Accuracy for iteration {run + 1}: {final_test_accuracy:.4f}")




🔄 Iteration 1/10
step number  0
step number  1
step number  2
step number  3
step number  4
step number  5
step number  6
step number  7
step number  8
step number  9
initial_params= [0.24837610892904338, -0.7768388163753946, 0.564232634605594, 0.16373665057686304, 0.20290017081123157, 0.6036656315488492, 0.29718760078993434]


KeyboardInterrupt: 

In [ ]:
mean_accuracy = np.mean(test_accuracies)
std_accuracy = np.std(test_accuracies)

print(f"Mean: {mean_accuracy:.4f}, Std: {std_accuracy:.4f}")

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch.nn import Linear, Sequential, ReLU, Dropout
from sklearn.metrics import accuracy_score

class HighPassConv(MessagePassing):
    def __init__(self, in_channels, out_channels, amp=0.5, **kwargs):
        kwargs.setdefault('aggr', 'add')
        super().__init__(**kwargs)
        self.amp = amp
        self.lin = Linear(in_channels, out_channels)
        self.bias = nn.Parameter(torch.Tensor(out_channels))
        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        nn.init.zeros_(self.bias)

    def forward(self, x, edge_index, edge_weight=None):
        x = self.lin(x)
        out = self.propagate(edge_index, x=x, edge_weight=edge_weight)
        out = self.amp * x - out
        return out

    def message(self, x_j, edge_weight):
        return x_j if edge_weight is None else edge_weight.view(-1, 1) * x_j

class Augmenter(nn.Module):
    def __init__(self, in_dim, hidden_dim, amp=0.5):
        super().__init__()
        self.conv1 = HighPassConv(in_dim, hidden_dim, amp=amp)
        self.conv2 = HighPassConv(hidden_dim, hidden_dim, amp=amp)
        self.mlp_edge_model = Sequential(
            Dropout(0.5),
            Linear(hidden_dim, hidden_dim * 2),
            ReLU(),
            Dropout(0.5),
            Linear(hidden_dim * 2, 1)
        )

    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        src, dst = edge_index[0], edge_index[1]
        edge_emb = x[src] + x[dst]
        return torch.sigmoid(self.mlp_edge_model(edge_emb))

class HeterophilicNodeClassifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, num_classes, test_indices, initial_preds=None):
        super().__init__()
        self.augmenter = Augmenter(in_dim, hidden_dim)
        self.gnn_high = HighPassConv(in_dim, hidden_dim)
        self.offset_mlp = Sequential(
            Linear(in_dim, hidden_dim),
            ReLU(),
            Linear(hidden_dim, hidden_dim)
        )
        self.final_mlp = Sequential(
            Linear(hidden_dim, hidden_dim),
            ReLU(),
            Linear(hidden_dim, num_classes)
        )

        self.test_indices = test_indices  # Store test indices

        # Store initial predictions but only for test indices (not trained)
        if initial_preds is not None:
            self.initial_preds = F.one_hot(initial_preds.long(), num_classes=num_classes).float()
        else:
            self.initial_preds = None

    def forward(self, x, edge_index, train_indices):
        edge_weights = self.augmenter(x, edge_index)
        h = self.gnn_high(x, edge_index, edge_weights)
        offset = self.offset_mlp(x)
        logits = self.final_mlp(h + offset)

        # If initial_preds exists, add it **only for test nodes**
        if self.initial_preds is not None:
            logits[self.test_indices] = logits[self.test_indices] + self.initial_preds

        return F.log_softmax(logits, dim=1)

# Training and Evaluation Code
def train_and_evaluate(model, data, train_indices, test_indices, initial_preds, epochs=2000, lr=0.01):
    criterion = nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        logits = model(data.x, data.edge_index, train_indices)
        loss = criterion(logits[train_indices], data.y[train_indices])
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        logits = model(data.x, data.edge_index, train_indices)
        predicted_classes = torch.argmax(logits, dim=1)
        test_predictions = predicted_classes[test_indices]
        final_accuracy = accuracy_score(data.y[test_indices].cpu().numpy(), test_predictions.cpu().numpy())

    return final_accuracy, test_predictions





In [ ]:
# Compute initial test predictions
initial_preds, initial_accuracy = predictclass(data, train_idx, test_idx, best_a1, best_a2, best_a3, best_a4, best_a5,best_a6)

# Ensure `initial_preds` is a tensor and on the correct device
initial_preds = initial_preds.clone().detach()  # Ensures a proper detached copy


print(f"📊 Initial Classification Accuracy: {initial_accuracy:.4f}")

# Create the model
model = HeterophilicNodeClassifier(
    in_dim=data.x.shape[1],
    hidden_dim=32,
    num_classes=len(set(data.y.tolist())),test_indices=test_indices, initial_preds=initial_preds
).to(data.x.device)  # Ensure model is on the correct device

# 🔄 Train the model before inference
final_accuracy_before_djgnn, test_predictions_before_djgnn = train_and_evaluate(
    model, data, train_idx, test_idx, initial_preds, epochs=100, lr=0.01
)

print(len(test_predictions_before_djgnn),len(test_idx))

# ✅ Model evaluation after training
model.eval()
#with torch.no_grad():
 #   logits = model(data.x, data.edge_index, train_idx, test_idx, initial_test_preds=initial_preds)
 #   predicted_classes = torch.argmax(logits, dim=1)


print(f"🎯  Classification Accuracy after running kdd2023: {final_accuracy_before_djgnn:.4f}")
#print(test_predictions)
#print(len(initial_preds))
#print(len(test_predictions))

#print(len(val_idx))


In [ ]:
test_accuracies = []
strategy_used = []

num_iterations = 10

for run in range(num_iterations):
    print(f"\n🔄 Iteration {run + 1}/{num_iterations}")

    indices = np.arange(data.num_nodes)
    train_val_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=run, shuffle=True)
    train_idx, val_idx = train_test_split(train_val_idx, test_size=0.25, random_state=run)  # 0.25 * 0.8 = 0.2

    train_idx = torch.tensor(train_idx, dtype=torch.long).contiguous()
    val_idx = torch.tensor(val_idx, dtype=torch.long).contiguous()
    test_idx = torch.tensor(test_idx, dtype=torch.long).contiguous()

    train_data = data.subgraph(train_idx)
    val_data = data.subgraph(val_idx)

    # Step 1: Hyperparameter tuning on training data
    best_params_random, best_val_random = random_search(predictclass, train_data, num_samples=15, num_splits=5)
    best_params_bayes, best_val_bayes = bayesian_optimization(predictclass, train_data, num_calls=25, initial_params=best_params_random)

    best_params = best_params_bayes if best_val_bayes >= best_val_random else best_params_random
    best_a1, best_a2, best_a3, best_a4, best_a5, best_a6, best_a7 = best_params

    # Step 2: Run original model on validation set
    _, val_accuracy_original = predictclass(data, train_idx, val_idx, best_a1, best_a2, best_a3, best_a4, best_a5, best_a6, best_a7)
    print(f"📊 Validation Accuracy (Original Model): {val_accuracy_original:.4f}")

    # Step 3: Compute initial preds to feed into kdd2023 model
    initial_preds, _ = predictclass(data, train_idx, val_idx, best_a1, best_a2, best_a3, best_a4, best_a5, best_a6, best_a7)
    initial_preds = initial_preds.clone().detach()

    # Step 4: Define and train kdd2023 model
    model = HeterophilicNodeClassifier(
        in_dim=data.x.shape[1],
        hidden_dim=32,
        num_classes=len(set(data.y.tolist())),
        test_indices=val_idx,
        initial_preds=initial_preds
    ).to(data.x.device)

    val_accuracy_kdd2023, _ = train_and_evaluate(model, data, train_idx, val_idx, initial_preds, epochs=100, lr=0.01)
    print(f"🧠 Validation Accuracy (kdd2023 Model): {val_accuracy_kdd2023:.4f}")

    # Step 5: Choose better strategy based on validation accuracy
    if val_accuracy_kdd2023 > val_accuracy_original:
        chosen_strategy = "Used kdd2023"
        print("✅ Using kdd2023 model on test set.")

        # Recompute initial_preds for test
        initial_preds_test, _ = predictclass(data, train_idx, test_idx, best_a1, best_a2, best_a3, best_a4, best_a5, best_a6, best_a7)
        initial_preds_test = initial_preds_test.clone().detach()

        # Recreate model with test_idx and initial_preds_test
        model = HeterophilicNodeClassifier(
            in_dim=data.x.shape[1],
            hidden_dim=32,
            num_classes=len(set(data.y.tolist())),
            test_indices=test_idx,
            initial_preds=initial_preds_test
        ).to(data.x.device)

        test_accuracy, _ = train_and_evaluate(model, data, train_idx, test_idx, initial_preds_test, epochs=100, lr=0.01)

    else:
        chosen_strategy = "Skipped kdd2023"
        print("🚫 Skipping kdd2023. Using original model on test set.")

        _, test_accuracy = predictclass(data, train_idx, test_idx, best_a1, best_a2, best_a3, best_a4, best_a5, best_a6, best_a7)

    # Record results
    test_accuracies.append(test_accuracy)
    strategy_used.append(chosen_strategy)

    print(f"🎯 Final Test Accuracy: {test_accuracy:.4f} | Strategy: {chosen_strategy}")

# Summary
print("\n📊 Summary of Results:")
for i in range(num_iterations):
    print(f"Iteration {i+1}: Test Accuracy = {test_accuracies[i]:.4f}, Strategy = {strategy_used[i]}")

print(f"\n📈 Average Test Accuracy: {sum(test_accuracies) / len(test_accuracies):.4f}")

import numpy as np  # Make sure this is at the top if not already imported

print(f"📉 Standard Deviation of Test Accuracies: {np.std(test_accuracies):.4f}")



🔄 Iteration 1/10
step number  0
step number  1
step number  2
step number  3
step number  4
step number  5
step number  6
step number  7
step number  8
step number  9
step number  10
step number  11
step number  12
step number  13
step number  14
initial_params= [0.22349722318651266, -0.7121533595290257, 0.7727272829991054, 0.2592863096841016, 0.5455576587399936, 0.7387728671118222, 0.15545123261498978]
📊 Validation Accuracy (Original Model): 0.8919


In [17]:
import numpy as np  # Make sure this is at the top if not already imported

print(f"📉 Standard Deviation of Test Accuracies: {np.std(test_accuracies):.4f}")


📉 Standard Deviation of Test Accuracies: 0.0439


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


class DJGNN(nn.Module):
    def __init__(self, in_dim, hidden_dim, num_classes, test_indices, initial_preds, n_jumps=5, dropout=0.5):
        super(DJGNN, self).__init__()
        self.n_jumps = n_jumps
        self.dropout = dropout
        self.att = nn.Parameter(torch.ones(n_jumps + 1))  # Attention weights for jumps
        self.sm = nn.Softmax(dim=0)

        # Store initial predictions (converted to one-hot)
        self.test_indices = test_indices
        self.initial_preds = nn.Parameter(F.one_hot(initial_preds, num_classes).float(), requires_grad=False)

        # GCN layers
        self.convs = nn.ModuleList()
        for _ in range(n_jumps):
            self.convs.append(GCNConv(in_dim, hidden_dim))

        # Extra convolution layers
        self.conv_extra = GCNConv(in_dim, hidden_dim)
        self.bn_extra = nn.BatchNorm1d(hidden_dim)
        self.conv_extra2 = GCNConv(hidden_dim, hidden_dim)
        self.bn_extra_2 = nn.BatchNorm1d(hidden_dim)

        # Final classifier
        self.classify = nn.Linear(hidden_dim * (n_jumps + 1) + num_classes, num_classes)  # Extra input for initial preds

    def forward(self, x, edge_index):
      mask_attentions = self.sm(self.att)

      # Extra convolutions
      extra_conv = self.conv_extra(x, edge_index).relu()
      print(f"extra_conv after conv_extra: {extra_conv.shape}")
      extra_conv = self.bn_extra(extra_conv)
      print(f"extra_conv after bn_extra: {extra_conv.shape}")
      extra_conv = F.dropout(extra_conv, p=self.dropout, training=self.training)
      extra_conv = self.conv_extra2(extra_conv, edge_index).relu()
      extra_conv = self.bn_extra_2(extra_conv)
      extra_conv = extra_conv * mask_attentions[-1]
      print(f"extra_conv after conv_extra2 & mask: {extra_conv.shape}")

      # Jumping knowledge mechanism: collect outputs from each conv layer
      z_s = []
      for i, conv in enumerate(self.convs):
          z = conv(x, edge_index).relu() * mask_attentions[i]
          print(f"z from conv {i}: {z.shape}")
          z_s.append(z)

      # Concatenate all jump outputs with extra_conv
      final_z = torch.cat(z_s + [extra_conv], dim=1)
      print(f"✅ final_z shape BEFORE concatenation: {final_z.shape}")  # Expect [num_nodes, feature_dim]

      # Debug: Print test indices shape and the slice from final_z
      print(f"✅ self.test_indices shape: {self.test_indices.shape}")  # Expect [num_test_samples]
      test_slice = final_z[self.test_indices]
      print(f"✅ final_z[self.test_indices] shape: {test_slice.shape}")  # Expect [num_test_samples, feature_dim]

      # Ensure self.initial_preds is a LongTensor
      num_classes = self.classify.out_features  # Get number of classes
      initial_preds_long = self.initial_preds.long().squeeze()
      print(f"initial_preds_long shape after squeeze: {initial_preds_long.shape}")

      # Clamp indices to valid range
      initial_preds_long = torch.clamp(initial_preds_long, min=0, max=num_classes - 1)

      # Convert to one-hot encoding
      one_hot_preds = F.one_hot(initial_preds_long, num_classes=num_classes).float()
      print(f"✅ one_hot_preds shape immediately after one_hot: {one_hot_preds.shape}")

      # If one_hot_preds still has an extra dimension, print its dimensions explicitly
      for idx, dim in enumerate(one_hot_preds.shape):
          print(f"one_hot_preds dim {idx}: {dim}")

      # Attempt to remove extra dimensions by squeezing only the dims of size 1
      one_hot_preds = one_hot_preds.squeeze()
      print(f"✅ one_hot_preds shape after squeeze(): {one_hot_preds.shape}")

      # Check dimensions explicitly
      if one_hot_preds.ndim != 2:
          print(f"❌ one_hot_preds still has {one_hot_preds.ndim} dimensions; expected 2 dimensions!")

      # Now, try concatenation and catch errors with more prints
      try:
          final_z[self.test_indices] = torch.cat((final_z[self.test_indices], one_hot_preds), dim=1)
      except RuntimeError as e:
          print(f"❌ ERROR: Shape mismatch at concatenation!")
          print(f"🔍 final_z[self.test_indices] shape: {final_z[self.test_indices].shape}")
          print(f"🔍 one_hot_preds shape: {one_hot_preds.shape}")
          raise e  # Re-raise after printing shapes

      # Dropout and classification
      final_z = F.dropout(final_z, p=self.dropout, training=self.training)
      logits = self.classify(final_z).log_softmax(dim=-1)
      return logits






In [ ]:
def train_djgnn(model, data, train_idx, test_idx, initial_preds_djgnn, epochs=100, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
    criterion = nn.NLLLoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        logits = model(data.x, data.edge_index)
        loss_supervised = criterion(logits[train_idx], data.y[train_idx])

        # 🔥 NEW: Consistency Loss to prevent drastic changes
        confidence_threshold = 0.85  # Change only high-confidence nodes
        high_confidence_mask = logits[test_idx].softmax(dim=1).max(dim=1).values > confidence_threshold

        # Apply consistency loss only to confident predictions
        filtered_loss = F.cross_entropy(logits[test_idx][high_confidence_mask], initial_preds_djgnn[high_confidence_mask])
        loss = loss_supervised + 0.1 * filtered_loss



        loss.backward()
        optimizer.step()

        # Compute accuracy on test set
        model.eval()
        with torch.no_grad():
            test_logits = model(data.x, data.edge_index)
            test_preds = test_logits[test_idx].argmax(dim=1)
            test_correct = (test_preds == data.y[test_idx]).sum().item()
            test_acc = test_correct / len(test_idx)

    #    print(f"🧐 Epoch {epoch+1:02d} | Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")

       # if epoch == 0 or (epoch + 1) % 5 == 0:
       #     print(f"🔍 Logits Before Argmax (First 5 Samples):\n{test_logits[test_idx][:5]}")

    return test_preds, test_acc



In [ ]:
import torch

# Ensure `initial_preds_djgnn` is a tensor and detached
initial_preds_djgnn = test_predictions_before_djgnn.clone().detach().long()  # Explicitly convert to LongTensor
initial_preds = initial_preds.clone().detach().long()

# Get the number of unique classes from `data.y`
num_classes = data.y.max().item() + 1  # `max() + 1` ensures it captures all labels

print(f"📊 Number of Test Samples: {len(initial_preds_djgnn)}")
print(f"🔢 Number of Unique Classes: {num_classes}")

# 🚀 Initialize DJ-GNN model with test indices & initial predictions
djgnn_model = DJGNN(
    in_dim=data.x.shape[1],
    hidden_dim=32,
    num_classes=num_classes,  # Use the correct `num_classes`
    test_indices=torch.tensor(test_idx, dtype=torch.long, device=data.x.device),  # Ensure `test_idx` is a tensor
    initial_preds=initial_preds_djgnn,  # Pass initial predictions (already `LongTensor`)
    n_jumps=5,  # Use 5 hops for diffusion
    dropout=0.5
).to(data.x.device)  # Ensure it's on the correct device

# 🚀 Train DJ-GNN and get new predictions
new_preds, final_accuracy_after_djgnn = train_djgnn(
    djgnn_model, data, train_idx, test_idx, initial_preds_djgnn, epochs=500, lr=0.01
)

# 🎯 Compare accuracies before & after DJ-GNN
print("\n🔥 Final Results:")
print(f"🎯 Accuracy BEFORE DJ-GNN: {final_accuracy_before_djgnn:.4f}")
print(f"🚀 Accuracy AFTER DJ-GNN: {final_accuracy_after_djgnn:.4f}")

# 🔍 Sample Predictions Before & After DJ-GNN (Print first 10 for debugging)
print("\n🔍 Sample Predictions Before & After DJ-GNN:")
for i in range(min(10, len(test_idx))):  # Ensure we don't go out of bounds
    print(f"🧐 Sample {i+1} | Before: {initial_preds_djgnn[i].item()} | After: {new_preds[i].item()}")

